In [ ]:
%pip install -q chi

In [ ]:
from chi import server, context, lease
import os, time

context.version = "1.0" 
context.choose_project()
context.choose_site(default="CHI@TACC")

In [ ]:
l = lease.get_lease(f"node-gpu-project51") 
l.show()

In [ ]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-gpu-project51", 
    reservation_id=l.node_reservations[0]["id"],
    image_name="CC-Ubuntu24.04-CUDA"
)
s.submit(idempotent=True)

In [ ]:
s.associate_floating_ip()

In [ ]:
s.refresh()
s.check_connectivity()

In [ ]:
s.refresh()
s.show(type="widget")

In [ ]:
s.execute("git clone --recurse-submodules https://github.com/care-ai-mlops/care-companion.git")

## Docker Setup

In [ ]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh")
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER")

## Setup AMD GPU

In [ ]:
s.execute("sudo apt update; wget https://repo.radeon.com/amdgpu-install/6.3.3/ubuntu/noble/amdgpu-install_6.3.60303-1_all.deb")
s.execute("sudo apt -y install ./amdgpu-install_6.3.60303-1_all.deb; sudo apt update")

In [ ]:
s.execute("amdgpu-install -y --usecase=dkms")

In [ ]:
s.execute("sudo apt -y install rocm-smi")

In [ ]:
s.execute("sudo usermod -aG video,render $USER")

In [ ]:
s.execute("sudo reboot")
time.sleep(30)

In [ ]:
s.refresh()
s.check_connectivity()

In [ ]:
s.execute("rocm-smi")

In [ ]:
s.execute("sudo apt -y install cmake libncurses-dev libsystemd-dev libudev-dev libdrm-dev libgtest-dev")
s.execute("git clone https://github.com/Syllo/nvtop")
s.execute("mkdir -p nvtop/build && cd nvtop/build && cmake .. -DAMDGPU_SUPPORT=ON && sudo make install")